7.22 Dota version

Dota skills json file

In [48]:
import pandas as pd

In [144]:
import itertools

In [49]:
import urllib.request
import requests
from bs4 import BeautifulSoup

In [50]:
import urllib.request
import json
from urllib.request import urlopen

content = urlopen("https://raw.githubusercontent.com/odota/dotaconstants/master/build/abilities.json")

with urlopen("https://raw.githubusercontent.com/odota/dotaconstants/master/build/abilities.json") as json_file:
    data = json.load(json_file)

In [51]:
heroes_name_fixed = pd.read_csv('C:/Users/anhqu/OneDrive/Desktop/QAs/QA github folder/dota2_heroes.csv')

In [52]:
heroes_name_list = heroes_name_fixed['Heroes'].tolist()

In [53]:
[value for key, value in data.items() if 'slardar' in key.lower()]

[{'dname': 'Guardian Sprint',
  'behavior': ['No Target', 'Instant Cast', None],
  'desc': 'Slardar slithers ahead, moving significantly faster and passing through units. Slardar has unlocked max speed and grants movement speed while in the river.',
  'attrib': [{'key': 'bonus_speed',
    'header': 'BONUS MOVE SPEED:',
    'value': ['20%', '28%', '36%', '44%']},
   {'key': 'duration', 'header': 'DURATION:', 'value': '10'},
   {'key': 'river_speed',
    'header': 'RIVER SPEED:',
    'value': ['5', '20', '35', '50'],
    'generated': True},
   {'key': 'puddle_regen', 'header': 'SCEPTER HP REGEN:', 'value': '25'},
   {'key': 'puddle_armor', 'header': 'SCEPTER BONUS ARMOR:', 'value': '12'},
   {'key': 'puddle_status_resistance',
    'header': 'SCEPTER STATUS RESISTANCE:',
    'value': '40%'},
   {'key': 'river_speed_tooltip',
    'header': 'BONUS MOVE SPEED IN RIVER:',
    'value': ['25%', '48%', '66%', '94%']}],
  'cd': '17',
  'img': '/apps/dota2/images/abilities/slardar_sprint_md.png'},

In [54]:
# remove duplicates and keep the list order 
# http://www.martinbroadhurst.com/removing-duplicates-from-a-list-while-preserving-order-in-python.html

def unique(sequence):
    seen = set()
    return [x for x in sequence if not (x in seen or seen.add(x))]

what im trying to do so far is to websrap dota page theory about hero counters. 

In theory, each heroes will have counter heroes and items

In [92]:
def get_heroes_matchup_info(hero_name, heroes_mentioned):
    
    # Part 1: clean up the information collected from the webite  
    # filter non type element in the list
    heroes_mentioned = list(filter(None, heroes_mentioned)) 

    # cut short the list to only relevant heroes
    heroes_mentioned_clean = heroes_mentioned[heroes_mentioned.index('#Good_against...')+2 : 
                                              heroes_mentioned.index('/Special:Categories') -1]

    # remove '/' i the list
    heroes_mentioned_clean = [heroes_mentioned[i].strip('/') for i in 
                              range(len(heroes_mentioned))]    

    # remove duplicates in the list    
    heroes_mentioned_clean = unique(heroes_mentioned_clean)

    # remove skills and items (can think more about this later)
    heroes_mentioned_clean1 = []

    for i in range(len(heroes_mentioned_clean)):
        if '#' not in heroes_mentioned_clean[i]:
            heroes_mentioned_clean1.append(heroes_mentioned_clean[i])

    # filter only heroes        
    heroes_mentioned_clean2 = [heroes_mentioned_clean1[i] for i in 
                               range(len(heroes_mentioned_clean1)) if 
                               heroes_mentioned_clean1[i] in heroes_name_list]

    # Part 2: populate the hero's matchup info   
    # In the page, heroes that counters the hero well is in red  
    heroes_countered1 = []

    productDivs = soup.findAll('div', attrs={'style': 'margin-bottom:5px; box-shadow:0px 0px 2px 4px red;'})
    for div in productDivs:
        heroes_countered1.append(div.find('a')['href'].strip('/'))

    # In the page, heroes that the hero counters well is in green       
    heroes_counter1 = []

    productDivs_chartreuse = soup.findAll('div', attrs={'style': 'margin-bottom:5px; box-shadow:0px 0px 2px 4px chartreuse;'})
    productDivs_green = soup.findAll('div', attrs={'style': 'margin-bottom:5px; box-shadow:0px 0px 2px 4px green;'})
    productDivs = productDivs_chartreuse + productDivs_green
    for div in productDivs:
        heroes_counter1.append(div.find('a')['href'].strip('/'))

    # In the page, heroes good with are list in blue     
    heroes_good_with1 = []

    productDivs = soup.findAll('div', attrs={'style': 'margin-bottom:5px; box-shadow:0px 0px 2px 4px skyblue;'})
    for div in productDivs:
        heroes_good_with1.append(div.find('a')['href'].strip('/'))

    # In the page, great countered heroes are mentioned in red, then there are some other heroes mentioned before the page moves on to the next section
    # last hero mentioned in red 
    last_red = heroes_mentioned_clean2.index(heroes_countered1[-1])
    # first hero mentioned in green
    first_green = heroes_mentioned_clean2.index(heroes_counter1[0])
    # last hero mentioned in green
    last_green = heroes_mentioned_clean2.index(heroes_counter1[-1])

    # first hero mentioned in blue
    first_blue = heroes_mentioned_clean2.index(heroes_good_with1[0])
    # last hero mentioned in blue
    last_blue = heroes_mentioned_clean2.index(heroes_good_with1[-1])

    # Create list of heroes matchups    
    # Create countered level 2
    if last_red+1 == first_green:
        heroes_countered2 = []
    else:     
        heroes_countered2 = heroes_mentioned_clean2[last_red+1 : first_green]

    # Create counter level 2
    if last_green+1 == first_blue: 
        heroes_counter2 = []
    else:
        heroes_counter2 = heroes_mentioned_clean2[last_green+1 : first_blue]

    # Create good with level 2
    heroes_good_with2 = heroes_mentioned_clean2[last_blue+1 :]

    # Part 3: export the info 
    # to an object
    heroes_matchup_info = {'heroes_countered1': heroes_countered1, 
                           'heroes_countered2': heroes_countered2,
                           'heroes_counter1': heroes_counter1, 
                           'heroes_counter2': heroes_counter2,
                           'heroes_good_with1': heroes_good_with1,
                           'heroes_good_with2': heroes_good_with2}
    
    return heroes_matchup_info

get all the link in gamepedia page

In [133]:
r = requests.get('https://dota2.gamepedia.com/Matchups')
soup = BeautifulSoup(r.text)

base_url = 'https://dota2.gamepedia.com/Matchups'
heroes_counters_link = []

for link in soup.find_all('a'):
    url = link.get('href')
    if url and '/Counters' in url:
        heroes_counters_link.append(url)
        
heroes_counters_link = list(sorted(set(heroes_counters_link)))

In [141]:
heroes_matchup_dict = {}

for i in range(len(heroes_counters_link)):
    r = requests.get('https://dota2.gamepedia.com' + heroes_counters_link[i])
    soup = BeautifulSoup(r.text)

    hero_name = heroes_counters_link[i].strip('/')[:-9]

    heroes_mentioned = []
    for link in soup.findAll('a'):
        heroes_mentioned.append(link.get('href')) 
    
    heroes_matchup_dict[hero_name] = get_heroes_matchup_info(hero_name, heroes_mentioned)

list of all heroes and items are mentioned in the counter page of the hero, we need to rank great against, normal against, item against and skills against

In [138]:
for i in range(len(heroes_matchup_dict['Abaddon']['heroes_countered1'])):
   heroes_matchup_dict['Abaddon']['heroes_countered1'][i] = list([heroes_matchup_dict['Abaddon']['heroes_countered1'][i], 
         len(heroes_matchup_dict['Abaddon']['heroes_countered1']) - i]) 

In [ ]:
heroes_matchup_pair = list(itertools.permutations(heroes_name_list,2))

In [195]:
heroes_matchup_pair_add_score = heroes_matchup_pair

for i in range(len(heroes_matchup_pair)):
    if heroes_matchup_pair[i][1] in heroes_matchup_dict[heroes_matchup_pair[i][0]]['heroes_countered1']:
        heroes_matchup_pair_add_score.append(list([heroes_matchup_pair[i], -1]))
    if heroes_matchup_pair[i][1] in heroes_matchup_dict[heroes_matchup_pair[i][0]]['heroes_countered2']:
        heroes_matchup_pair_add_score.append(list([heroes_matchup_pair[i], -0.5]))
    if heroes_matchup_pair[i][1] in heroes_matchup_dict[heroes_matchup_pair[i][0]]['heroes_counter1']:
        heroes_matchup_pair_add_score.append(list([heroes_matchup_pair[i], 1]))
    if heroes_matchup_pair[i][1] in heroes_matchup_dict[heroes_matchup_pair[i][0]]['heroes_counter2']:
        heroes_matchup_pair_add_score.append(list([heroes_matchup_pair[i], 0.5]))
        

In [196]:
heroes_matchup_pair_add_score

[[('Abaddon', 'Ancient_Apparition'), -1],
 [('Abaddon', 'Anti-Mage'), 0.5],
 [('Abaddon', 'Axe'), -1],
 [('Abaddon', 'Bane'), 1],
 [('Abaddon', 'Batrider'), 1],
 [('Abaddon', 'Beastmaster'), 1],
 [('Abaddon', 'Bloodseeker'), 1],
 [('Abaddon', 'Brewmaster'), -1],
 [('Abaddon', 'Chen'), 1],
 [('Abaddon', 'Death_Prophet'), 1],
 [('Abaddon', 'Doom'), -1],
 [('Abaddon', 'Faceless_Void'), 0.5],
 [('Abaddon', 'Huskar'), 0.5],
 [('Abaddon', 'Juggernaut'), 0.5],
 [('Abaddon', 'Legion_Commander'), 0.5],
 [('Abaddon', 'Lina'), 0.5],
 [('Abaddon', 'Lion'), 0.5],
 [('Abaddon', 'Mars'), -1],
 [('Abaddon', 'Mirana'), 1],
 [('Abaddon', 'Monkey_King'), 0.5],
 [('Abaddon', 'Necrophos'), 1],
 [('Abaddon', 'Night_Stalker'), -0.5],
 [('Abaddon', 'Nyx_Assassin'), -0.5],
 [('Abaddon', 'Ogre_Magi'), 0.5],
 [('Abaddon', 'Outworld_Devourer'), -1],
 [('Abaddon', 'Phoenix'), 0.5],
 [('Abaddon', 'Pudge'), 0.5],
 [('Abaddon', 'Shadow_Demon'), -1],
 [('Abaddon', 'Shadow_Shaman'), 0.5],
 [('Abaddon', 'Silencer'), -1]

import pandas as pd

pd.read_excel('C:/Users/anhqu/OneDrive/Desktop/QAs/QA github folder/dota2_heroes.xlsx')